<a href="https://colab.research.google.com/github/hncpyj/DataAnalytics/blob/main/wpgCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Python crawling

In [18]:
import os, sys
import sqlite3
import traceback
import logging
logger = logging.getLogger('wpg_logger')
logging.basicConfig(encoding='utf-8', level=logging.INFO, format='%(levelname)s:%(name)s:%(asctime)s:%(message)s', filename='./logger.log', filemode='a')
logger.info('[Start]')
# 현재 경로 확인
# path=os.getcwd()
# print(path)

rootPath="/content/drive/MyDrive/supremeCourt"
sqlDir="/sql"
attachDir="/attachment/"
dbFileName="wpg_file.db"
dbFilePath = rootPath + "/" + sqlDir + "/" + dbFileName


In [35]:
logger.info('[createDB]')
## DB 접속 및 테이블 생성
def createDB(sqlDirPath):
  dbFilePath = sqlDirPath + "/" + dbFileName
  logger.info('[createDB]dbFilePath: ' + dbFilePath)
  # DB 접속 및 생성
  conn = sqlite3.connect(dbFilePath, isolation_level=None)
  logger.info('[createDB] conn success')
  # SQL 테이블 생성문
  createWpgSql = "CREATE TABLE IF NOT EXISTS wpgFile ([id] INTEGER PRIMARY KEY AUTOINCREMENT, [title] TEXT, [auth] TEXT, [reg_dt] TEXT, [file_path] TEXT)"
  logger.info('[createDB]createWpgSql: ' + createWpgSql)
  # 테이블 생성
  try:
    cur = conn.cursor()
    logger.info('[createDB] cur success')
    cur.execute(createWpgSql)
    logger.info('[createDB] execute completed')
  except sqlite3.Error as e:
    logger.error('[sqlite3.Error] SQLite error: %s' % (' '.join(e.args)))
    logger.error('[sqlite3.Error] Exception class is: ' + e.__class__)
    logger.error('[sqlite3.Error] SQLite traceback: ')
    exc_type, exc_value, exc_tb = sys.exc_info()
    logger.error('[sqlite3.Error] '+ traceback.format_exception(exc_type, exc_value, exc_tb))


In [36]:
logger.info('[makeDir]')
# 폴더 체크 후 없으면 생성
def makeDir(rootPath, dir, category): 
  # root 경로에 원하는 폴더 이름 합치기
  dirPath = rootPath + "/" + dir
  # sqlDirPath에 폴더가 있는지 확인
  if os.path.exists(dirPath):
    logger.info('[makeDir] exists makeDirPath : '+ dirPath)
    # 폴더가 있는 것이 확인되면 파일 체크
    if category=="s":
      createDB(dirPath)
  else:
    # folderPath에 원하는 폴더가 없으면 생성 시도
    try:
      os.makedirs(dirPath)
      logger.info('[makeDir] makedirs success : ' + dirPath)
      # 폴더를 만들고 파일 생성
      if category=="s":
        createDB(dirPath)
    except OSError:
      logger.info('[makeDir] Error: Creating directory. ' + dirPath)
      print('Error: Creating directory. ' +  dirPath)



In [37]:
################################
# logic
logger.info('[logic]logic start')
# 파일 생성
makeDir(rootPath, sqlDir, "s")

In [38]:
logger.info('[checkDB]')
## DB 접속 및 테이블 생성
def checkDB(dbFilePath):
  logger.info('[checkDB]dbFilePath: ' + dbFilePath)
  # DB 접속 및 생성
  conn = sqlite3.connect(dbFilePath, isolation_level=None)
  logger.info('[checkDB] conn success')
  # SQL 테이블 생성문
  checkWpgSql = ("select * from sqlite_master where type='table'")
  logger.info('[checkDB]SELECT WpgSql: ' + checkWpgSql)
  # 테이블 생성
  try:
    cur = conn.cursor()
    logger.info('[checkDB] cur success')
    cur.execute(checkWpgSql)
    logger.info('[checkDB] execute completed')
    rows = cur.fetchall()
    for row in rows:
      logger.info('[checkDB] select rows : ' + str(row))
      print(str(rows))
  except sqlite3.Error as e:
    logger.error('[sqlite3.Error] SQLite error: %s' % (' '.join(e.args)))
    logger.error('[sqlite3.Error] Exception class is: ' + e.__class__)
    logger.error('[sqlite3.Error] SQLite traceback: ')
    exc_type, exc_value, exc_tb = sys.exc_info()
    logger.error('[sqlite3.Error] '+ traceback.format_exception(exc_type, exc_value, exc_tb))


checkDB(dbFilePath)

[('table', 'wpgFile', 'wpgFile', 2, 'CREATE TABLE wpgFile ([id] INTEGER PRIMARY KEY AUTOINCREMENT, [title] TEXT, [auth] TEXT, [reg_dt] TEXT, [file_path] TEXT)'), ('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')]
[('table', 'wpgFile', 'wpgFile', 2, 'CREATE TABLE wpgFile ([id] INTEGER PRIMARY KEY AUTOINCREMENT, [title] TEXT, [auth] TEXT, [reg_dt] TEXT, [file_path] TEXT)'), ('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')]


In [39]:
#첨부파일 디렉토리 체크
makeDir(rootPath, attachDir, "a")

In [44]:
import json, requests
from bs4 import BeautifulSoup


url = "https://www.scourt.go.kr/portal/dcboard/DcNewsListAction.work?gubun=44"
response = requests.get(url)

# print(response.url)

if response.status_code == 200:
    for k, v in response.headers.items():
        header = '{}: {}'.format(k, v)
        # print(str(header))
    html = response.text
    # print("html : " + html)

    soup = BeautifulSoup(html, 'html.parser')

    # TotalPage Count 구하기
    result = soup.find('div', {'class': 'result'})
    result = result.text.strip()
    resultarr = result.split(" ")

    sResult = resultarr[2].replace("(", "")
    sResult = sResult.replace(")", "")
    sResult = sResult.replace("page", "")
    sResultArr = sResult.split("/")

    totPage = int(sResultArr[1])  # totalpage
    urls = {}

    # 첫번째 페이지 리스트 구하기
    # table = soup.find('table', {'class': 'tableHor'})
    # trs = table.find_all('tr')
    # for idx, tr in enumerate(trs):
    #     if idx > 0:
    #         tds = tr.find_all('td')
    #         sequence = tds[0].text.strip()
    #         clickevent = tds[1].find('a').attrs
    #         clickevent = str(clickevent['onclick'])
    #         clickevent = clickevent[8:24]
    #         pararmArr = clickevent.split(",")
    #         seqnum = pararmArr[0].replace("'", "")
    #         bubcode = pararmArr[1].replace("'", "")
    #         seqnum = str(seqnum).strip()
    #         bubcode = str(bubcode).strip()
    #         # print("seqnum : " + seqnum + ", bubcode : " + bubcode)
    #         # surl = str(surl[0:5])
    #         # print(surl.attrs)
    #         # title = surl[0].text.strip()
    #         writer = tds[2].text.strip()
    #         iSeq = int(sequence)
    #         # self.calcul_data[date].update({"현재가": current_price})

    #         # 리스트의 seqnm, bubcode 를 dictionary에 담기
    #         urls[seqnum] = {}
    #         urls[seqnum].update({"seqnum": seqnum})
    #         urls[seqnum].update({"bubcode": bubcode})
    # print(urls)
    # print("sequence : " + str(sequence) + ", title : " + str(title) + ", writer : " + str(writer))

    # 2페이지 부터 리스트 구하기
    for pageNum in range(200, 201):
      print("pageNum : " + str(pageNum))
      tempUrl = url + "&pageIndex=" + str(pageNum)
      logger.info('[callList]SELECT tempUrl: ' + tempUrl)
      print(tempUrl)
      rs = requests.get(tempUrl)
      if rs.status_code == 200:
        temphtml = rs.text
        # print("html : " + html)
        tempsoup = BeautifulSoup(temphtml, 'html.parser')
        temptable = tempsoup.find('table', {'class':'tableHor'})
        temptrs = temptable.find_all('tr')
        for idx, tr in enumerate(temptrs):
          if idx > 0:
            tds = tr.find_all('td')
            sequence = tds[0].text.strip()
            clickevent = tds[1].find('a').attrs
            clickevent = str(clickevent['onclick'])
            clickevent = clickevent[8:24]
            pararmArr = clickevent.split(",")
            seqnum = pararmArr[0].replace("'", "")
            bubcode = pararmArr[1].replace("'", "")
            seqnum = str(seqnum).strip()
            bubcode = str(bubcode).strip()

            #리스트의 seqnm, bubcode 를 dictionary에 담기
            urls[seqnum] = {}
            urls[seqnum].update({"seqnum":seqnum})
            urls[seqnum].update({"bubcode":bubcode})
            logger.info('[callList] urls sequence : ' + str(sequence))
            logger.info('[callList] urls bubcode : ' + str(bubcode))
            # print("sequence : " + str(sequence) + ", title : " + str(title) + ", writer : " + str(writer))
# print(urls)

pageNum : 200
https://www.scourt.go.kr/portal/dcboard/DcNewsListAction.work?gubun=44&pageIndex=200


In [45]:


## DB insert
def insertDB(dbFilePath):
  logger.info('[insertDB]dbFilePath: ' + dbFilePath)
  # DB 접속 및 생성
  conn = sqlite3.connect(dbFilePath, isolation_level=None)
  logger.info('[insertDB] conn success')
  # SQL 테이블 생성문
  # checkWpgSql = ("select * from sqlite_master where type='table'")
  # logger.info('[insertDB]SELECT WpgSql: ' + checkWpgSql)
  # 테이블 생성
  try:
    cur = conn.cursor()
    logger.info('[insertDB] cur success')

    # 디테일 페이지 들어간 후 제목, 작성자, 작성일 파일경로 가져와서 인서트
    for seqnum in urls.keys():
        seq = urls[seqnum]["seqnum"]
        bubcode = urls[seqnum]["bubcode"]
        # print("seq = " + seq + ", bubcode = " + bubcode)
        url2 = "https://www.scourt.go.kr/portal/dcboard/DcNewsViewAction.work?gubun=44"
        dUrl = url2 + "&pageSize=5&pageIndex=1&seqnum=" + seq + "&cbub_code= " + bubcode + "&searchOption=&searchWord="
        # print(dUrl)
        dRs = requests.get(dUrl)
        if dRs.status_code == 200:
            dHtml = dRs.text
            dsoup = BeautifulSoup(dHtml, 'html.parser')
            contents = dsoup.find('div', {'class': 'contentIn'})
            dTable = contents.find_all('table')
            temp = dTable[0]
            dTrs = temp.find_all('tr')
            
            title_td=dTrs[0].find('td')
            title=title_td.text.strip()
            sTitle=str(title).replace('"', '')
            print("title : " + sTitle)
            reg_tds=dTrs[1].find_all('td')
            auth=reg_tds[0].text.strip()
            sAuth=str(auth)
            print("auth : " + sAuth)
            reg_dt=reg_tds[1].text.strip()
            sReg_dt=str(reg_dt)
            print("reg_dt : " + str(reg_dt))
            prefilepath=dTrs[3].find('td')
            prefilepath=prefilepath.find_all('a')
            logger.info('[insertDB] prefilepath' + str(prefilepath))
            if  prefilepath:
              filepath_attr=prefilepath[0].attrs
              sfilepath = str(filepath_attr['href'])
            else:
              filepath_attr=""
              sfilepath = ""
            print("filepath : " + sfilepath)

            insertWpgSql = ('INSERT INTO wpgFile (title, auth, reg_dt, file_path) VALUES ("' + sTitle + '","' + sAuth + '","' + sReg_dt + '","' + sfilepath+'")')
            logger.info('[insertDB] insert query' + str(insertWpgSql))
            cur.execute(insertWpgSql)
            logger.info('[insertDB] execute completed' + str(seqnum))
            # for idx, dtr in enumerate(dTrs):
                # if idx > 0:
                #     tds = dtr.find_all('td')
                #     print("tds = " + str(tds))
                    # sequence = tds[0].text.strip()
                    # print(sequence)

  except sqlite3.Error as e:
    logger.error('[sqlite3.Error] SQLite error: %s' % (' '.join(e.args)))
    logger.error('[sqlite3.Error] Exception class is: ' + e.__class__)
    logger.error('[sqlite3.Error] SQLite traceback: ')
    exc_type, exc_value, exc_tb = sys.exc_info()
    logger.error('[sqlite3.Error] '+ traceback.format_exception(exc_type, exc_value, exc_tb))


insertDB(dbFilePath)


title : [행정][손실보상] 도시정비법상 재개발사업의 공람공고일 이후에 주점 영업을 양수한 원고가 자신이 실질사업주임을 주장하면서 손실보상을 받아야 함을 주장한 사안에서, 원고는 영업손실보상 대상을 정하는 기준일 이후에 영업을 양수한 자에 불과하여 영업손실보상 대상이 될 수 없는 자이고, 설령 해당 주점의 전전 업주가 공람공고일 이전부터 영업을 해오던 사람이었다고 하더라도 이러한 결론이 달라지지 않는다고&nbsp;본 판결(2020구합71901)
auth : 서울행정법원
reg_dt : 2022-03-14
filepath : https://file.scourt.go.kr//AttachDownload?file=1647242956545_162916.pdf&path=003&downFile=서울행정법원-2020구합71901.pdf
title : [행정][일반] 원고가 국가연구개발사업의 연구책임자로서 학생연구원 인건비 공동관리계좌를 사용되었음을 이유로 15년간의 참여제한처분 및 산학협력단에 연구비 환수처분을 한 사안에서, 공동관리계좌가 사용되어 온 사실이&nbsp;인정되고 원고가 몰랐다고 보기는 어려우므로 처분사유 자체는 인정되지만, 원고가 실제로 관여한 바는 없고 실제로 인건비 용도 외로는 사용된 적이 없음에도, 원고에게 각 과제별(3개)로&nbsp;상한에 해당하는 각 5년씩의 처분을 하여, 합계 15년간의 참여제한처분 및 전액 환수처분을 한 것은 비례원칙에 반하여 위법하다고 본&nbsp;판결(2020구합78391)
auth : 서울행정법원
reg_dt : 2022-03-14
filepath : https://file.scourt.go.kr//AttachDownload?file=1647242713798_162513.pdf&path=003&downFile=서울행정법원-2020구합78391.pdf
title : [행정][일반] 라돈침대 관련 불기소 사건의 고소인들 중 1명이었던 원고가, 전문가자문회의 등을 비롯한 전반적인 수시기록에 대하여 정보공개를 구한 사건에서, 

In [54]:
logger.info('[selectDB]')
## DB 접속 및 테이블 생성
def selectDB(dbFilePath):
  logger.info('[selectDB]dbFilePath: ' + dbFilePath)
  # DB 접속 및 생성
  conn = sqlite3.connect(dbFilePath, isolation_level=None)
  logger.info('[selectDB] conn success')
  # SQL 테이블 생성문
  # selectWpgSql = ("delete from wpgFile where id >2450 and id<2461")
  # selectWpgSql = ("select * from wpgFile where title ='[행정][일반] 라돈침대 관련 불기소 사건의 고소인들 중 1명이었던 원고가, 전문가자문회의 등을 비롯한 전반적인 수시기록에 대하여 정보공개를 구한 사건에서, 공공의 안전과 시민들의 건강이 관련된 공익적 성격을 갖는 사건의 특성 등에 비추어&nbsp;일반적인 형사사건보다 알권리가 보다 중요한 영역이라는 이유로 수사기록에 대한 정보공개의 범위를 비교적 폭넓게 인정한 판결(2021구합62393)' ")
  selectWpgSql = ("select * from wpgFile")
  logger.info('[selectDB]SELECT WpgSql: ' + selectWpgSql)
  # 테이블 생성
  try:
    cur = conn.cursor()
    logger.info('[selectDB] cur success')
    cur.execute(selectWpgSql)
    logger.info('[selectDB] execute completed')
    rows = cur.fetchall()
    for row in rows:
      logger.info('[selectDB] select rows : ' + str(row))
      print(str(row))
  except sqlite3.Error as e:
    logger.error('[sqlite3.Error] SQLite error: %s' % (' '.join(e.args)))
    logger.error('[sqlite3.Error] Exception class is: ' + e.__class__)
    logger.error('[sqlite3.Error] SQLite traceback: ')
    exc_type, exc_value, exc_tb = sys.exc_info()
    logger.error('[sqlite3.Error] '+ traceback.format_exception(exc_type, exc_value, exc_tb))

selectDB(dbFilePath)

(461, '[행정][손실보상] 도시정비법상 재개발사업의 공람공고일 이후에 주점 영업을 양수한 원고가 자신이 실질사업주임을 주장하면서 손실보상을 받아야 함을 주장한 사안에서, 원고는 영업손실보상 대상을 정하는 기준일 이후에 영업을 양수한 자에 불과하여 영업손실보상 대상이 될 수 없는 자이고, 설령 해당 주점의 전전 업주가 공람공고일 이전부터 영업을 해오던 사람이었다고 하더라도 이러한 결론이 달라지지 않는다고&nbsp;본 판결(2020구합71901)', '서울행정법원', '2022-03-14', 'https://file.scourt.go.kr//AttachDownload?file=1647242956545_162916.pdf&path=003&downFile=서울행정법원-2020구합71901.pdf')
(462, '[행정][일반] 원고가 국가연구개발사업의 연구책임자로서 학생연구원 인건비 공동관리계좌를 사용되었음을 이유로 15년간의 참여제한처분 및 산학협력단에 연구비 환수처분을 한 사안에서, 공동관리계좌가 사용되어 온 사실이&nbsp;인정되고 원고가 몰랐다고 보기는 어려우므로 처분사유 자체는 인정되지만, 원고가 실제로 관여한 바는 없고 실제로 인건비 용도 외로는 사용된 적이 없음에도, 원고에게 각 과제별(3개)로&nbsp;상한에 해당하는 각 5년씩의 처분을 하여, 합계 15년간의 참여제한처분 및 전액 환수처분을 한 것은 비례원칙에 반하여 위법하다고 본&nbsp;판결(2020구합78391)', '서울행정법원', '2022-03-14', 'https://file.scourt.go.kr//AttachDownload?file=1647242713798_162513.pdf&path=003&downFile=서울행정법원-2020구합78391.pdf')
(463, '[행정][일반] 라돈침대 관련 불기소 사건의 고소인들 중 1명이었던 원고가, 전문가자문회의 등을 비롯한 전반적인 수시기록에 대하여 정보공개를 구한 사건에서, 공공의 안전과 시민들의 건강이 관련된 공익적 성격을 갖는 사건의

KeyboardInterrupt: ignored

In [10]:
def download(url, filePathName):
    with open(filePathName, "wb") as file:   # open in binary mode
        response = get(url)               # get request
        file.write(response.content)      # write to file

In [11]:
from requests import get  


logger.info('[fileDown]')
## DB 접속 및 테이블 생성
def fileDown(dbFilePath, attaPath):
  logger.info('[selectFile]dbFilePath: ' + dbFilePath)
  # DB 접속 및 생성
  conn = sqlite3.connect(dbFilePath, isolation_level=None)
  logger.info('[selectFile] conn success')
  # SQL 테이블 생성문
  selectWpgSql = ("select file_path from wpgFile")
  logger.info('[selectFile]SELECT WpgSql: ' + selectWpgSql)
  # 파일 이름 잘라서 놓을 변수 생성
  fileName=""

  # 테이블 생성
  try:
    cur = conn.cursor()
    logger.info('[selectFile] cur success')
    cur.execute(selectWpgSql)
    logger.info('[selectFile] execute completed')
    rows = cur.fetchall()
    for row in rows:
      url=str(row).replace("('", "").replace(",", "").replace("')", "")
      print("url : "+ url)
      logger.info('[selectFile] select rows : ' + url)
      fileName = url.split('=')
      sfileName = fileName[-1]
      logger.info('[selectFile] fileName : ' + sfileName)
      filePathName = attaPath+sfileName
      logger.info('[selectFile] filePathName : ' + filePathName)
      print(sfileName)
      download(url, filePathName)
      logger.info('[selectFile] select rows : ' + str(row))
  except sqlite3.Error as e:
    logger.error('[sqlite3.Error] SQLite error: %s' % (' '.join(e.args)))
    logger.error('[sqlite3.Error] Exception class is: ' + e.__class__)
    logger.error('[sqlite3.Error] SQLite traceback: ')
    exc_type, exc_value, exc_tb = sys.exc_info()
    logger.error('[sqlite3.Error] '+ traceback.format_exception(exc_type, exc_value, exc_tb))


attaPath=rootPath+attachDir
fileDown(dbFilePath, attaPath)

url : https://file.scourt.go.kr//AttachDownload?file=1647242956545_162916.pdf&path=003&downFile=서울행정법원-2020구합71901.pdf
서울행정법원-2020구합71901.pdf
url : https://file.scourt.go.kr//AttachDownload?file=1647242713798_162513.pdf&path=003&downFile=서울행정법원-2020구합78391.pdf
서울행정법원-2020구합78391.pdf
url : https://file.scourt.go.kr//AttachDownload?file=1647242155748_161555.pdf&path=003&downFile=서울행정법원-2021구합62393.pdf
서울행정법원-2021구합62393.pdf
url : https://file.scourt.go.kr//AttachDownload?file=1647242060937_161420.pdf&path=003&downFile=서울행정법원-2021구합54231.pdf
서울행정법원-2021구합54231.pdf
url : https://file.scourt.go.kr//AttachDownload?file=1647241931097_161211.pdf&path=003&downFile=서울행정법원-2021구합51089.pdf
서울행정법원-2021구합51089.pdf
url : https://file.scourt.go.kr//AttachDownload?file=1647241842999_161042.pdf&path=003&downFile=서울행정법원-2020구합88183.pdf
서울행정법원-2020구합88183.pdf
url : https://file.scourt.go.kr//AttachDownload?file=1647241758097_160918.pdf&path=003&downFile=서울행정법원-2020구합85931.pdf
서울행정법원-2020구합85931.pdf
url : 